# Discovery of brown dwarfs mining the 2MASS and SDSS databases

F. Jiménez-Esteban¹, K. A. Lutz², E. Solano¹

1. Spanish Virtual Observatory
2. Université de Strasbourg, CNRS, Observatoire Astronomique de Strasbourg, UMR 7550, F-67000, Strasbourg, France

This tutorial is based on the EURO-VO tutorial of the same name: http://www.euro-vo.org/sites/default/files/documents/tutorial-brown-dwarfs_2019Apr.pdf .

***

## Introduction 

Brown dwarfs are objects occupying the gap between the least massive stars and the most massive planets. They are intrinsically faint objects. Hence, heir detection is not straightforward and, in fact, was almost impossible until the advent of global surveys at deep optical and near-infrared bands like SDSS, 2MASS or DENIS. We propose here to mine the 2MASS point source catalogue (2MASS-PSC) and SDSS-DR9 databases to identify T-type brown dwarfs through an appropriate combination of colors in the optical and the infrared, an approach that perfectly fits into the Virtual Observatory.

In this use case, we explore different ways to do the same tasks with different VO tools. These tasks
include:
 1. obtaining data from the SDSS and 2MASS catalogues in a given sky region,
 2. crossmatching the results of these searches,
 3. filtering the resulting table for brown dwarfs, and
 4. verifying our sample of brown dwarfs.

Software packages needed for this tutorial are Aladin, TOPCAT, STILTS and Python (or you may just use the Binder in your web browser).

In [1]:
# Standard Library
from pathlib import Path

# Astronomy tools
from astropy import units as u
from astropy.coordinates import SkyCoord

# Access astronomical databases
import pyvo
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch

# Sky visualization
from ipyaladin import Aladin

## Step #1: Discovery

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

We start by searching for the 2MASS point source catalogue (2MASS-PSC) and the SDSS data release 9 catalogue (SDSS DR9). To find and query catalogues, there are two packages we can use: pyVO and astroquery. We will show pyVO for searching the 2MASS-PSC and astroquery for SDSS DR9. 

With the pyVo package we are using the TAP service of VizieR. TAP is a Virtual Observatory protocol to query databases of tables. The language we use to compile the query is called ADQL and is similar to SQL. For more details on ADQL see also the dedicated VO tutorial (http://docs.g-vo.org/adql-gaia/html/). 

In [2]:
tap_vizier = pyvo.dal.TAPService("https://tapvizier.cds.unistra.fr/TAPVizieR/tap")

query = """
SELECT  * 
FROM tables
WHERE description LIKE '%2MASS%Cutri%'
"""

mass_psc_set = tap_vizier.search(query).to_table()
mass_psc_set["table_name", "description"]

table_name,description
object,object
J/ApJ/569/23/table1,"Optical polarimetry of 2MASS Red QSOs ( Smith P.S., Schmidt G.D., Hines D.C., Cutri R.M., Nelson B.O.)"
J/ApJ/780/92/2mass,"2MASS JHKs photometry (table5) ( Szabo R., Ivezic Z., Kiss L.L., Kollath Z., Jones L., Sesar B., Becker A.C., Davenport J.R.A., Cutri R.M.)"
II/281/2mass6x,"2MASS-6X catalog ( Cutri R.M., Skrutskie M.F., Van Dyk S., et al.)"
J/ApJ/564/421/table3,"2MASS IDR2 Sources Unconfirmed in Second Epoch Imaging ( Burgasser A.J., Kirkpatrick J.D., Brown M.E., Reid I.N., Burrows A., Liebert J., Matthews K., Gizis J.E., Dahn C.C., Monet D.G., Cutri R.M., Skrutskie M.F.)"
II/246/out,"2MASS Point Source Catalogue, output (on a total of 470,992,970 sources) ( Cutri R.M., Skrutskie M.F., Van Dyk S., et al.)"
VII/233/xsc,"*The 2MASS Extended Catalog (2MASX) ( Skrutskie M.F., Cutri R.M., Stiening R., et al.)"
II/241/out,"2MASS Point Source Catalogue, 2MASS 2000 Second Incremental Release ( Skrutskie M.F., Schneider S.E., Stiening R., Strom S.E., Weinberg M.D., Beichman C., Chester T., Cutri R., Lonsdale C., Elias J., Elston R., Capps R., Carpenter J., Huchra J., Liebert J., Monet D., Price S., Seitzer P.)"
J/ApJ/569/23/table3,"Galactic interstellar polarization detected in the fields of 2MASS AGNs ( Smith P.S., Schmidt G.D., Hines D.C., Cutri R.M., Nelson B.O.)"
J/ApJ/569/23/table2,"Optical polarimetry of other AGN found by 2MASS ( Smith P.S., Schmidt G.D., Hines D.C., Cutri R.M., Nelson B.O.)"


As mentioned above we are interested in the 2MASS-pSC, which has the table_name `II/246/out` in VizieR. Before we move on to construct a more complicated query, we have a quick look at the 2MASS-PSC table and all the available columns. We do so by downloading the first 5 rows:

In [3]:
query = 'SELECT TOP 5 * FROM "II/246/out" '
mass_psc_head = tap_vizier.search(query).to_table()
mass_psc_head

RAJ2000,DEJ2000,errMaj,errMin,errPA,2MASS,Jmag,Jcmsig,e_Jmag,Jsnr,Hmag,Hcmsig,e_Hmag,Hsnr,Kmag,Kcmsig,e_Kmag,Ksnr,Qflg,Rflg,Bflg,Cflg,Ndet,prox,pxPA,pxCntr,Xflg,Aflg,Cntr,Hemis,Date,Scan,GLON,GLAT,Xscan,JD,Jpsfchi,Hpsfchi,Kpsfchi,Jstdap,e_Jstdap,Hstdap,e_Hstdap,Kstdap,e_Kstdap,edgeNS,edgeEW,edge,dup,use,opt,Dopt,PAopt,Bmag,Rmag,Nopt,extKey,scanKey,coaddKey,coadd
deg,deg,arcsec,arcsec,deg,,mag,mag,mag,,mag,mag,mag,,mag,mag,mag,,,,,,,arcsec,deg,,,,,,,,deg,deg,arcsec,d,,,,mag,mag,mag,mag,mag,mag,arcsec,arcsec,,,,,arcsec,deg,mag,mag,,,,,
float64,float64,float32,float32,int16,object,float32,float32,float32,float64,float32,float32,float32,float64,float32,float32,float32,float64,object,object,object,object,object,float32,int16,int64,int16,int16,int64,str1,object,int16,float32,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int16,object,int16,int16,str1,float32,int16,float32,float32,int16,int32,int32,int32,int16
44.996055,0.005565,0.17,0.16,76,02595905+0000200,16.376,0.097,0.097,11.3,15.77,0.139,0.14,8.2,15.258,0.141,0.141,7.6,ABB,222,111,000,060606,44.5,230,1290669337,0,0,1290669325,s,1998-09-28,112,176.951,-48.902,-138.9,2451084.8062,0.75,1.21,1.13,16.651,0.221,15.434,0.134,15.478,0.434,521,114,ne,1,1,U,0.7,156,17.9,16.9,1,--,69347,1594959,9
45.004857,0.019806,0.06,0.06,90,03000116+0001113,12.529,0.021,0.024,389.8,11.954,0.029,0.03,273.7,11.874,0.027,0.029,172.5,AAA,222,111,000,666666,4.8,330,1290669298,0,0,1290669300,s,1998-09-28,112,176.945,-48.885,-170.6,2451084.8062,1.15,1.31,1.97,12.508,0.019,11.91,0.025,11.852,0.008,469,82,ne,1,1,U,1.5,72,15.0,13.8,1,--,69347,1594959,9
45.004193,0.020956,0.06,0.06,90,03000100+0001154,14.845,0.055,0.056,46.2,14.223,0.077,0.077,33.9,14.016,0.055,0.055,24.0,AAA,222,111,ccc,666666,4.8,150,1290669300,0,0,1290669298,s,1998-09-28,112,176.943,-48.885,-168.2,2451084.8062,0.86,0.89,1.11,13.889,0.101,13.035,0.126,12.97,0.043,465,84,ne,1,1,0,--,--,--,--,0,--,69347,1594959,9
44.995074,0.038204,0.38,0.31,0,02595881+0002175,16.746,0.133,0.134,8.0,15.814,0.139,0.14,7.8,16.125,0.324,0.324,3.4,BBD,222,111,000,060606,70.2,152,1290669298,0,0,1290669276,s,1998-09-28,112,176.914,-48.88,-135.4,2451084.8062,1.22,0.84,0.69,17.047,0.414,15.5,0.229,17.513,1.964,403,117,ne,0,1,0,--,--,--,--,0,--,69347,1594959,9
44.963851,0.043587,0.22,0.17,95,02595132+0002369,16.476,0.112,0.113,10.3,16.057,0.175,0.175,6.3,15.564,--,--,--,BCU,220,110,000,060600,89.4,207,1290669295,0,0,1290669271,s,1998-09-28,112,176.875,-48.898,-23.0,2451084.8062,1.13,1.07,--,17.113,0.34,18.739,3.527,--,--,384,229,ne,1,1,U,0.4,161,18.7,17.7,1,--,69347,1594959,9


As in the original tutorial, we only want to get data within a 14arcmin radius of RA=08h30m00s, Dec=01d30m00s. ADQL has a function, which allows to run an astronomical cone search query. From the quick inital inspection of the table we now know that we will need the columns `RAJ2000` and `DEJ2000` to only get sources at a certain location. Since we want to get colours for our Brown Dwarf candidates, we will also want to get the `Jmag`, `Hmag` and `Kmag` columns. For a different analysis you might want to choose other columns. 

In [4]:
coord = SkyCoord(ra="08:30:00", dec="01:30:00", unit=(u.hourangle, u.deg))

query = """
SELECT * 
FROM \"II/246/out\" as tm_psc 
WHERE 1=CONTAINS(POINT(\'ICRS\', tm_psc.RAJ2000, tm_psc.DEJ2000),
                 CIRCLE(\'ICRS\', {:.3f}, {:.3f}, 14/60))
""".format(
    coord.ra.value,
    coord.dec.value,
)

mass_psc = tap_vizier.search(query).to_table()
mass_psc

RAJ2000,DEJ2000,errMaj,errMin,errPA,2MASS,Jmag,Jcmsig,e_Jmag,Jsnr,Hmag,Hcmsig,e_Hmag,Hsnr,Kmag,Kcmsig,e_Kmag,Ksnr,Qflg,Rflg,Bflg,Cflg,Ndet,prox,pxPA,pxCntr,Xflg,Aflg,Cntr,Hemis,Date,Scan,GLON,GLAT,Xscan,JD,Jpsfchi,Hpsfchi,Kpsfchi,Jstdap,e_Jstdap,Hstdap,e_Hstdap,Kstdap,e_Kstdap,edgeNS,edgeEW,edge,dup,use,opt,Dopt,PAopt,Bmag,Rmag,Nopt,extKey,scanKey,coaddKey,coadd
deg,deg,arcsec,arcsec,deg,,mag,mag,mag,,mag,mag,mag,,mag,mag,mag,,,,,,,arcsec,deg,,,,,,,,deg,deg,arcsec,d,,,,mag,mag,mag,mag,mag,mag,arcsec,arcsec,,,,,arcsec,deg,mag,mag,,,,,
float64,float64,float32,float32,int16,object,float32,float32,float32,float64,float32,float32,float32,float64,float32,float32,float32,float64,object,object,object,object,object,float32,int16,int64,int16,int16,int64,str1,object,int16,float32,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int16,object,int16,int16,str1,float32,int16,float32,float32,int16,int32,int32,int32,int16
127.509154,1.269173,0.2,0.15,87,08300219+0116090,15.963,0.095,0.096,12.5,15.546,0.121,0.122,9.0,14.93,0.144,0.144,8.1,ABB,222,111,000,160606,14.2,340,804040589,0,0,804040599,s,2000-01-30,58,223.554,22.408,178.5,2451573.6815,0.89,1.01,0.92,15.939,0.235,15.899,0.319,14.69,0.124,4589,74,sw,0,1,0,--,--,--,--,0,--,43956,1010984,221
127.507779,1.272875,0.12,0.11,86,08300186+0116223,15.594,0.083,0.084,17.6,15.563,0.117,0.117,8.9,15.272,0.181,0.182,5.9,ABC,222,111,000,360606,14.2,160,804040599,0,0,804040589,s,2000-01-30,58,223.55,22.408,183.4,2451573.6814,1.64,0.82,1.47,15.432,0.153,15.81,0.424,14.992,0.114,4602,69,sw,0,1,U,0.2,270,17.1,16.5,1,--,43956,1010984,221
127.514167,1.277495,0.23,0.21,51,08300340+0116389,16.553,0.17,0.171,7.3,16.179,0.194,0.194,5.0,15.894,--,--,--,CCU,220,110,000,060600,28.4,234,804040589,0,0,804040577,s,2000-01-30,58,223.549,22.416,160.4,2451573.6814,0.69,1.58,--,16.335,0.139,18.014,3.36,--,--,4619,92,sw,0,1,U,0.0,13,18.0,17.3,1,--,43956,1010984,221
127.463251,1.271903,0.07,0.07,0,08295118+0116188,15.844,0.076,0.077,14.0,15.11,0.095,0.096,13.5,15.029,0.138,0.138,7.4,AAB,222,111,000,260606,41.0,160,804019559,0,0,804019587,s,2000-01-30,57,223.528,22.369,-87.9,2451573.6739,1.78,0.8,1.14,15.935,0.238,15.077,0.164,15.197,0.438,4593,164,se,0,1,U,1.0,281,20.0,18.4,1,--,43955,1010947,56
127.463023,1.2864,0.06,0.06,0,08295112+0117110,14.093,0.029,0.032,70.0,13.652,0.033,0.035,51.7,13.553,0.042,0.043,28.9,AAA,222,111,000,666635,52.2,179,804019587,0,0,804019616,s,2000-01-30,57,223.514,22.376,-87.0,2451573.674,0.75,0.97,0.91,14.077,0.023,13.576,0.05,13.692,0.14,4646,165,se,0,1,U,0.4,160,16.3,14.6,1,--,43955,1010947,56
127.48465,1.271192,0.06,0.06,0,08295631+0116162,13.848,0.026,0.029,87.8,13.177,0.02,0.022,80.0,12.932,0.023,0.024,51.1,AAA,222,111,000,666655,15.7,12,804019600,0,0,804019585,s,2000-01-30,57,223.54,22.387,-164.9,2451573.6739,0.88,0.87,0.95,13.813,0.042,13.143,0.051,13.009,0.056,4591,87,se,0,1,U,0.2,244,18.8,16.7,1,--,43955,1010947,56
127.485565,1.275456,0.2,0.17,4,08295653+0116316,16.26,0.112,0.112,9.5,15.908,0.172,0.172,6.5,15.264,0.184,0.184,6.0,BCC,222,111,000,160606,15.7,192,804019585,0,0,804019600,s,2000-01-30,57,223.536,22.39,-168.2,2451573.674,0.84,1.3,0.8,16.125,0.291,15.529,0.253,15.21,0.246,4606,84,se,0,1,U,0.4,232,18.1,17.4,1,--,43955,1010947,56
127.494339,1.280734,0.15,0.15,145,08295864+0116506,16.317,0.119,0.12,9.0,15.818,0.141,0.141,7.0,15.099,--,--,--,BBU,220,110,000,060600,36.9,239,804019600,0,0,804019605,s,2000-01-30,57,223.536,22.4,-199.8,2451573.674,0.96,1.62,--,16.264,0.088,16.219,0.553,--,--,4625,52,se,1,1,0,--,--,--,--,0,--,43955,1010947,56


Now that we have all the data we want from the 2MASS-PSC, we move on to query SDSS DR9 with astroquery. First we want to find out which tables are available:

In [6]:
catalog_list_sdss = Vizier.find_catalogs("SDSS DR9")
for catalog_ID, catalog in catalog_list_sdss.items():
    print(catalog_ID, ": ", catalog.description)

V/139 :  The SDSS Photometric Catalog, Release 9 (Adelman-McCarthy+, 2012)
J/A+A/547/L1 :  SDSS-III DR9 DLA catalogue (Noterdaeme+, 2012)
J/A+A/598/A92 :  New ultracool subdwarfs (Lodieu+, 2017)
J/A+A/616/A97 :  SDSS QSO DR7 and DR9 (D'Isanto+, 2018)
J/MNRAS/444/2456 :  Spectral galaxy pairs from SDSS DR9 (Yang+, 2014)
J/MNRAS/445/1331 :  White-dwarf + main-sequence binaries in SDSS DR9 (Li+, 2014)
J/MNRAS/450/905 :  New SNe in SDSS DR9 (Graur+, 2015)
J/MNRAS/458/3808 :  DR9-12 SDSS WDMS binaries (Rebassa-Mansergas+, 2016)
J/other/NewA/58.61 :  SDSS DR9 galaxy clusters optical catalog (Banerjee+, 2018)
J/other/RAA/17.62 :  Candidate members of 4 stellar streams (Li+, 2017)


We want to get data from the catalogue called `V/139`. As before let's have a look at a few rows (per default `Vizier.get_catalogs` gives you 50 rows) of the code to figure out, which ones are most interesting for us.

In [7]:
test_sdss = Vizier.get_catalogs("V/139")
print(test_sdss)
test_sdss[0]

TableList with 1 tables:
	'0:V/139/sdss9' with 21 column(s) and 50 row(s) 


mode,q_mode,cl,SDSS9,m_SDSS9,Im,RA_ICRS,DE_ICRS,ObsDate,Q,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,zsp
,,,,,,deg,deg,yr,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,
uint8,str1,uint8,str19,str1,str2,float64,float64,float64,uint8,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64
2,,6,J025955.29+000252.0,*,Im,44.980386,0.047779,2003.8093,3,23.450,0.580,22.345,0.101,21.948,0.104,22.044,0.163,21.474,0.404,--
2,,3,J025956.07+000300.0,*,Im,44.983649,0.050016,1998.8264,3,24.432,1.374,24.266,0.627,22.859,0.314,21.240,0.119,21.032,0.430,--
2,,3,J025956.06+000259.9,*,Im,44.983619,0.049995,2002.7581,3,25.964,0.601,24.082,0.489,23.250,0.415,21.666,0.146,20.346,0.189,--
2,,3,J025956.06+000259.7,*,Im,44.983618,0.049940,2003.7384,3,25.593,0.783,24.345,0.567,22.358,0.174,21.124,0.084,20.942,0.310,--
2,,3,J025956.06+000259.9,*,Im,44.983616,0.049976,2001.7179,3,23.097,0.818,23.954,0.721,21.839,0.188,20.827,0.102,20.139,0.238,--
2,,3,J025956.06+000259.9,*,Im,44.983614,0.049984,2007.8858,3,25.534,1.705,24.458,1.282,23.156,0.451,21.522,0.119,20.811,0.229,--
2,,3,J025956.06+000259.9,*,Im,44.983607,0.049985,2002.8508,3,24.403,1.152,23.370,0.273,22.722,0.261,21.445,0.148,20.530,0.285,--
2,,3,J025956.06+000259.9,*,Im,44.983606,0.049981,1998.7254,3,23.421,0.462,24.430,0.472,22.850,0.206,21.512,0.091,21.022,0.241,--


Again we want to get the coordinates of the source as well as magnitudes at all five SDSS bands (u, g, r, i  and z), and a classification (cl). Do restrict the query to returning these columns and to get more than 50 rows returned, we create a custom `Vizier` object and then use this object to again query within 14arcmin of RA=08h30m00s, Dec=01d30m00s.

In [8]:
custom_vizier = Vizier(
    row_limit=-1,
    columns=["RA_ICRS", "DE_ICRS", "umag", "gmag", "rmag", "imag", "zmag", "cl"],
)
coord = SkyCoord(ra="08:30:00", dec="01:30:00", unit=(u.hourangle, u.deg))
sdss_results = custom_vizier.query_region(coord, radius=14 * u.arcmin, catalog="V/139")
sdss_results

TableList with 1 tables:
	'0:V/139/sdss9' with 8 column(s) and 12405 row(s) 

So we have one table in the resulting lists of tables. It is called `V/139/sdss9`. Let's have a look at it:

In [9]:
sdss_results[0]

RA_ICRS,DE_ICRS,umag,gmag,rmag,imag,zmag,cl
deg,deg,mag,mag,mag,mag,mag,
float64,float64,float32,float32,float32,float32,float32,uint8
127.267366,1.512930,21.853,19.732,18.870,18.564,18.379,6
127.267372,1.512928,22.250,19.697,18.855,18.568,18.306,6
127.267372,1.512928,22.047,19.751,18.879,18.549,18.383,6
127.267727,1.495873,22.879,22.223,21.492,21.122,20.872,3
127.267754,1.495933,22.972,22.431,21.465,21.162,21.009,3
127.267831,1.500005,23.816,24.462,22.470,20.754,20.033,6
127.267836,1.495947,22.638,22.051,21.045,20.627,20.785,3
127.267853,1.499961,24.527,24.690,22.381,20.670,20.016,6


## Step #2: Cross-matching


[![Xmatch](https://custom-icon-badges.demolab.com/badge/Xmatch-gray.svg?logo=xmatch&logoColor=blue&logoWidth=20)](http://cdsxmatch.u-strasbg.fr/ "http://cdsxmatch.u-strasbg.fr/")

In this next step, we find common sources in the 2MASS-PSC and SDSS-DR9 catalogues. One simple way to do this using the CDS XMatch service, which can also be reached with `astroquery`. The most reliable way to quuery XMatch is by uploading one of the tables (which will also make sure we stay in our region of interest) and crossmatching it to a table available in VizieR. 

In [10]:
mass_psc.write("Data/2MASS_PSC.vot", format="votable", overwrite=True)
sdss_results[0].write("Data/SDSS_DR9.vot", format="votable", overwrite=True)

In [11]:
sdss_mass = XMatch.query(
    cat1=Path("Data/2MASS_PSC.vot").open(),
    cat2="vizier:V/139/sdss9",
    max_distance=4 * u.arcsec,
    colRA1="RAJ2000",
    colDec1="DEJ2000",
)
sdss_mass

angDist,RAJ2000,DEJ2000,errMaj,errMin,errPA,2MASS,Jmag,Jcmsig,e_Jmag,Jsnr,Hmag,Hcmsig,e_Hmag,Hsnr,Kmag,Kcmsig,e_Kmag,Ksnr,Qflg,Rflg,Bflg,Cflg,Ndet,prox,pxPA,pxCntr,Xflg,Aflg,Cntr,Hemis,Date,Scan,GLON,GLAT,Xscan,JD,Jpsfchi,Hpsfchi,Kpsfchi,Jstdap,e_Jstdap,Hstdap,e_Hstdap,Kstdap,e_Kstdap,edgeNS,edgeEW,edge,dup,use,opt,Dopt,PAopt,Bmag,Rmag,Nopt,extKey,scanKey,coaddKey,coadd,SDSS9,RAdeg,DEdeg,errHalfMaj,errHalfMin,errPosAng,umag,gmag,rmag,imag,zmag,e_umag,e_gmag,e_rmag,e_imag,e_zmag,objID,cl,q_mode,flags,Q,ObsDate,pmRA,e_pmRA,pmDE,e_pmDE,SpObjID,zsp,e_zsp,f_zsp,spType,spCl,subClass
float64,float64,float64,float64,float64,int64,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str3,int64,int64,str3,int64,float64,int64,int64,int64,int64,int64,str1,str10,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,str2,int64,int64,str1,float64,int64,float64,float64,int64,int64,int64,int64,int64,str19,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,str16,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
0.352101,127.509154,1.269173,0.2,0.15,87,08300219+0116090,15.963,0.095,0.096,12.5,15.546,0.121,0.122,9.0,14.93,0.144,0.144,8.1,ABB,222,111,000,160606,14.2,340,804040589,0,0,804040599,s,2000-01-30,58,223.554,22.408,178.5,2451573.6815,0.89,1.01,0.92,15.939,0.235,15.899,0.319,14.69,0.124,4589,74,sw,0,1,0,--,--,--,--,0,--,43956,1010984,221,J083002.21+011608.9,127.509248,1.269146,0.01,0.009,90,23.712,20.876,19.402,18.068,17.338,0.67,0.034,0.015,0.009,0.015,1237653664712163908,6,1,0000201010000110,3,2000.9155,-53,5,-43,5,0,--,--,--,--,--,--
0.152435,127.507779,1.272875,0.12,0.11,86,08300186+0116223,15.594,0.083,0.084,17.6,15.563,0.117,0.117,8.9,15.272,0.181,0.182,5.9,ABC,222,111,000,360606,14.2,160,804040599,0,0,804040589,s,2000-01-30,58,223.55,22.408,183.4,2451573.6814,1.64,0.82,1.47,15.432,0.153,15.81,0.424,14.992,0.114,4602,69,sw,0,1,U,0.2,270,17.1,16.5,1,--,43956,1010984,221,J083001.86+011622.2,127.507749,1.272845,0.002,0.002,90,18.394,17.213,16.806,16.674,16.623,0.014,0.004,0.004,0.005,0.01,1237653664712163358,6,1,0000001010000000,3,2000.9155,-3,3,2,3,0,--,--,--,--,--,--
0.217574,127.514167,1.277495,0.23,0.21,51,08300340+0116389,16.553,0.17,0.171,7.3,16.179,0.194,0.194,5.0,15.894,--,--,--,CCU,220,110,000,60600,28.4,234,804040589,0,0,804040577,s,2000-01-30,58,223.549,22.416,160.4,2451573.6814,0.69,1.58,--,16.335,0.139,18.014,3.36,--,--,4619,92,sw,0,1,U,0.0,13,18.0,17.3,1,--,43956,1010984,221,J083003.39+011638.8,127.514129,1.277448,0.003,0.003,90,19.513,18.189,17.714,17.559,17.485,0.028,0.006,0.006,0.007,0.016,1237653664712163741,6,1,0000001010000000,3,2000.9155,2,3,1,3,0,--,--,--,--,--,--
0.048922,127.463251,1.271903,0.07,0.07,0,08295118+0116188,15.844,0.076,0.077,14.0,15.11,0.095,0.096,13.5,15.029,0.138,0.138,7.4,AAB,222,111,000,260606,41.0,160,804019559,0,0,804019587,s,2000-01-30,57,223.528,22.369,-87.9,2451573.6739,1.78,0.8,1.14,15.935,0.238,15.077,0.164,15.197,0.438,4593,164,se,0,1,U,1.0,281,20.0,18.4,1,--,43955,1010947,56,J082951.17+011618.8,127.463238,1.271907,0.005,0.005,90,21.976,19.794,18.386,17.54,17.048,0.176,0.016,0.008,0.007,0.012,1237653664712098290,6,1,0000001010000000,3,2000.9155,-19,4,8,4,0,--,--,--,--,--,--
0.023156,127.463023,1.2864,0.06,0.06,0,08295112+0117110,14.093,0.029,0.032,70.0,13.652,0.033,0.035,51.7,13.553,0.042,0.043,28.9,AAA,222,111,000,666635,52.2,179,804019587,0,0,804019616,s,2000-01-30,57,223.514,22.376,-87.0,2451573.674,0.75,0.97,0.91,14.077,0.023,13.576,0.05,13.692,0.14,4646,165,se,0,1,U,0.4,160,16.3,14.6,1,--,43955,1010947,56,J082951.12+011711.0,127.463018,1.286396,0.001,0.001,90,17.223,15.796,15.259,15.074,14.993,0.008,0.003,0.003,0.003,0.005,1237653664712098015,6,1,0000201090020010,3,2000.9155,5,3,-5,3,0,--,--,--,--,--,--
0.102852,127.48465,1.271192,0.06,0.06,0,08295631+0116

For the SDSS DR9 table this action has given us all the columns. So before we move on, we restrict the table to the columns, we are actually interested in. 

In [12]:
sdss_mass = sdss_mass[
    "2MASS",
    "RAJ2000",
    "DEJ2000",
    "Jmag",
    "Hmag",
    "Kmag",
    "SDSS9",
    "RAdeg",
    "DEdeg",
    "umag",
    "gmag",
    "rmag",
    "imag",
    "zmag",
    "cl",
]

print(f"We are currently working with {len(sdss_mass)} sources")

We are currently working with 717 sources


## Step #3: Filtering
In this step we select all those sources from our cross-matched catalogue that have colours and fluxes as expected for brown dwarfs. 

The selection cirteria for Brown Dwarfs are:
 - `umag > 22.0` --> magnitude in u-band fainter than 22.0mag
 - `gmag > 22.2` --> magnitude in g-band fainter than 22.2mag
 - `Jmag - Hmag < 0.3` --> J-H band colour smaller than 0.3mag
 - `Hmag - Kmag < 0.3` --> H-K band colour smaller than 0.3mag

We now apply these selection criteria to our table of point sources. 

In [13]:
# Add columsn with J-H and H-K colour
sdss_mass["J-H"] = sdss_mass["Jmag"] - sdss_mass["Hmag"]
sdss_mass["H-K"] = sdss_mass["Hmag"] - sdss_mass["Kmag"]
# Limit sources to those that fulfil the selection criteria
index_bd = (
    (sdss_mass["umag"] > 22.0)
    & (sdss_mass["gmag"] > 22.2)
    & (sdss_mass["J-H"] < 0.3)
    & (sdss_mass["H-K"] < 0.3)
)
candidates = sdss_mass[index_bd]
print(f"We have {len(candidates)} candidates for Brown Dwarfs")

We have 2 candidates for Brown Dwarfs


In [14]:
candidates

2MASS,RAJ2000,DEJ2000,Jmag,Hmag,Kmag,SDSS9,RAdeg,DEdeg,umag,gmag,rmag,imag,zmag,cl,J-H,H-K
str16,float64,float64,float64,float64,float64,str19,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64
08304878+0128311,127.703265,1.47532,16.289,16.14,16.358,J083048.81+012831.0,127.703377,1.475293,25.322,24.514,25.492,25.878,19.401,3,0.1490000000000009,-0.21799999999999997
08304512+0134404,127.688007,1.577901,16.18,16.021,15.769,J083045.23+013443.5,127.688491,1.578759,24.004,22.299,21.03,20.478,20.53,6,0.15899999999999892,0.25200000000000067


Note that we are using slightly different selection criteria here in the notebook than in the original tutorial. The reason is the following. The CDS XMatch service uses VizieR tables with their default filtering. For the SDSS DR 9 table, VizieR will filter to only use primary sources and disregard secondary sources, which is not the case when downloading the VizieR table with e.g. TOPCAT. Now the primary SDSS source for this brown dwarf is classified as an extended source by the automated SDSS pipeline, probably because it is hardly detected and very faint. So if we remove all extended sources we also throw away the target of our studies. Hence, the filtering in this notebook is less strict to allow 

## Step #4: Verification of the Brown Dwarf candidates

### Visualization

[![Aladin](https://custom-icon-badges.demolab.com/badge/Aladin-gray.svg?logo=aladin&logoColor=purple&logoWidth=20)](https://aladin.cds.unistra.fr/aladin.gml "https://aladin.cds.unistra.fr/aladin.gml")

The ipyaladin package allows to run an instance of Aladin Lite in your Jupyter notebook. 

In [15]:
aladin1 = Aladin(
    survey="P/SDSS9/color",
    fov=0.03,
    target="{:.4f} {:.4f}".format(candidates["RAJ2000"][0], candidates["DEJ2000"][0]),
)
aladin1

Aladin(fov=0.03, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'o…

As with any instance of Aladin Lite you can zoom in and out by scrolling your mouse. In the top left corner of the widget, you may chose which survey to look at, add layers, search for sources, ... 

You can also add tables to the widget by:

In [16]:
aladin1.add_table(candidates)

If you now click on the small square indicating the location of the source, the widget will show the corresponding table values in the lower part of the widget. 

As you can see at one of the locations hardly anything is visible and at the location of the other brown dwarf candidate, there is a bright star. To further investigate these two sources, we now query Simbad for them. 

### Check in Simbad

[![Simbad](https://custom-icon-badges.demolab.com/badge/Simbad-gray.svg?logo=simbad&logoColor=lightblue&logoWidth=20)](https://simbad.u-strasbg.fr/simbad/ "https://simbad.u-strasbg.fr/simbad/")

In the Simbad database, an object type is assigned to every entry. So if this Brown Dwarf candidate is in the Simbad database, we might learn more about the object type:

In [17]:
# First candidate has an index 0
customSimbad = Simbad()
customSimbad.remove_votable_fields("coordinates")
customSimbad.add_votable_fields(
    "ra(2;A;ICRS;J2000;2000)",
    "dec(2;D;ICRS;J2000;2000)",
    "otype",
)
candidate_coord = SkyCoord(
    ra=candidates["RAJ2000"][0],
    dec=candidates["DEJ2000"][0],
    unit=u.deg,
)
candidate_simbad = customSimbad.query_region(candidate_coord, radius=10 * u.arcsec)
candidate_simbad

MAIN_ID,RA_2_A_ICRS_J2000_2000,DEC_2_D_ICRS_J2000_2000,OTYPE,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,
object,str13,str13,object,int32
2MASS J08304878+0128311,08 30 48.7836,+01 28 31.152,BrownD*,1


In [18]:
# First candidate has an index 1
customSimbad = Simbad()
customSimbad.remove_votable_fields("coordinates")
customSimbad.add_votable_fields(
    "ra(2;A;ICRS;J2000;2000)",
    "dec(2;D;ICRS;J2000;2000)",
    "otype",
)
candidate_coord = SkyCoord(
    ra=candidates["RAJ2000"][1],
    dec=candidates["DEJ2000"][1],
    unit=u.deg,
)
candidate_simbad = customSimbad.query_region(candidate_coord, radius=1 * u.arcmin)
candidate_simbad

/home/manon.marchand/miniconda3/envs/cds-tutos/lib/python3.10/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '8:30:45.12168 +1:34:40.4436': No astronomical object found :
  warnings.warn("Warning: The script line number %i raised "


Indeed the first candidate is classified as a brown dwarf. Within 10arcsec of the second source, no entry is found in Simbad although this is the brighter star. The reason for this might be the following: sources in Simbad are sources that have individually been discussed in the literature. If no papers have been published specifcially about this object, it will not appear in Simbad. However, as the object was included in the SDSS tables it is included in VizieR through the table it belongs to. 

If we now want to see, in which papers the Brown Dwarf 2MASS J08304878+0128311 was included, we may use the following query in Simbad.

In [19]:
tap_simbad = pyvo.dal.TAPService("https://simbad.cds.unistra.fr/simbad/sim-tap")

query = """
SELECT BIBCode, Journal, Title, "year", Volume,
       Page || '-' || Last_Page AS "Pages", DOI 
FROM ref JOIN has_ref ON oidbibref = oidbib 
         JOIN ident ON has_ref.oidref = ident.oidref 
WHERE id = '2MASS J08304878+0128311'
ORDER BY "year" DESC; 
"""

result = tap_simbad.search(query).to_table()
result

bibcode,journal,title,year,volume,Pages,doi
object,object,object,int16,int32,object,object
2020AJ....159..257B,AJ,The Hawaii infrared parallax program. IV. A comprehensive parallax survey of L0-T8 dwarfs with UKIRT.,2020,159,257-257,10.3847/1538-3881/ab84f4
2016ApJ...827...25B,ApJ,The orbit of the L dwarf + T dwarf spectral binary SDSS J080531.84+481233.0.,2016,827,25-25,10.3847/0004-637X/827/1/25
2016A&A...589A..49S,A&A,Photometric brown-dwarf classification. II. A homogeneous sample of 1361 L and T dwarfs brighter than J = 17.5 with accurate spectral types.,2016,589,49-49,10.1051/0004-6361/201527359
2016ApJ...817..112S,ApJ,A proper motion survey using the first sky pass of NEOWISE-reactivation data.,2016,817,,10.3847/0004-637X/817/2/112
2015ApJ...810..158F,ApJ,Fundamental parameters and spectral energy distributions of young and field age objects with masses spanning the stellar to planetary regime.,2015,810,,10.1088/0004-637X/810/2/158
2014ApJ...793...75R,ApJ,Strong brightness variations signal cloudy-to-clear transition of brown dwarfs.,2014,793,,10.1088/0004-637X/793/2/75
2013MNRAS.433.2054S,MNRAS,"NPARSEC: NTT Parallaxes of Southern Extremely Cool objects. Goals, targets, procedures and first results.",2013,433,2054-2063,10.1093/mnras/stt876
2013MNRAS.433..457B,MNRAS,"76 T dwarfs from the UKIDSS LAS: benchmarks, kinematics and an updated space density.",2013,433,457-497,10.1093/mnras/stt740
2012ApJ...752...56F,ApJ,The Brown Dwarf Kinematics Project (BDKP). III. Parallaxes for 70 ultracool dwarfs.,2012,752,,10.1088/0004-637X/752/1/56


For further reading you could now go to [ADS](https://ui.adsabs.harvard.edu/ "https://ui.adsabs.harvard.edu/") and search for the publications via their bibcode (enter in the search field `bibcode:...`) or their doi (`doi:...`).

To design similar queries for your own research, have a look at [Simbad database schema](https://simbad.cds.unistra.fr/simbad/tap/tapsearch.html "https://simbad.cds.unistra.fr/simbad/tap/tapsearch.html").